In [102]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import re

In [136]:
site='https://www.ndbc.noaa.gov/box_search.php?lat1=-35.8333&lat2=7&lon1=-55.20&lon2=-20&uom=M&ot=A&time=12'


In [137]:
resp=requests.get(site)
soup=BeautifulSoup(resp.text,'html.parser')


In [138]:
lines=soup.find_all('span', attrs={'style': 'background-color: #f0f8fe'})

In [139]:
lines=soup.find_all('span', attrs={'style': 'background-color: #f0f8fe'})

values=[]
for line in lines:
    line = line.get_text(strip=True)
    line = line.replace("B", " B")
    line = re.sub(" +", ",", line).split(",")
    values.append(np.array(line))


In [140]:
lines = soup.find_all('span', attrs={'style': 'background-color: #fffff0'})
for line in lines:
    line = line.get_text(strip=True)
    line = line.replace("B", " B")
    line = re.sub(" +", ",", line).split(",")
    values.append(np.array(line))


In [141]:
columns=['ID','T1','hour','LAT','LON','WDIR','WSPD','GST','WVHT','DPD','APD','MWD','PRES','PTDY','ATMP','WTMP','DEWP','VIS','TCC','TIDE','S1HT','S1PD','S1DIR','S2HT','S2PD','S2DIR']

In [142]:
start_date = datetime.utcnow()-timedelta(hours=12)
end_date = datetime.utcnow()

In [151]:
df = pd.DataFrame(values).iloc[: , :26]

In [152]:
df.columns = columns

In [153]:
df.hour = (df.hour.astype(int)/100).astype(int)

In [154]:
df['date_time'] = 0

In [172]:
def calculate_date(x):
    start_date = datetime.utcnow()-timedelta(hours=12)
    end_date = datetime.utcnow()
    if x >= end_date.hour + 2:
        value = start_date
        value = value.replace(hour=x)
    else:
        value = end_date
        value = value.replace(hour=x)
    
    value = value.replace(minute=0)
    value = value.replace(second=0)
    value = value.replace(microsecond=0)
    return value

In [173]:
df['date_time'] = df['hour'].apply(lambda x: calculate_date(x))

In [177]:
df.replace('-', np.nan, inplace=True)

In [186]:
df = df.astype(float, errors='ignore')

In [187]:
df = df[['date_time','LAT','LON','WDIR','WSPD','WVHT','DPD','MWD','PRES','ATMP','WTMP','DEWP','S1HT','S1DIR']]

In [190]:
df.columns = ['date_time', 'lat', 'lon', 'wdir', 'wspd', 'swvht', 'tp', 'wvdir', 'pres', 'atmp', 'sst', 'dewpt', 'swvht_swell', 'wvdir_swell']

In [191]:
df["institution"] = 'ndbc'
df["data_type"] = 'gts'

In [188]:
df.head()

,date_time,LAT,LON,WDIR,WSPD,WVHT,DPD,MWD,PRES,ATMP,WTMP,DEWP,S1HT,S1DIR
0,2021-09-10 19:00:00,1.2,-30.2,NaN,NaN,NaN,NaN,NaN,1012.0,NaN,NaN,NaN,NaN,NaN
1,2021-09-10 18:00:00,4.4,-44.5,NaN,NaN,NaN,NaN,NaN,1012.7,30.9,NaN,25.0,NaN,NaN
2,2021-09-10 18:00:00,-2.3,-44.2,50.0,8.2,NaN,NaN,NaN,1011.0,29.0,28.0,23.2,NaN,NaN
3,2021-09-10 18:00:00,-12.0,-35.6,90.0,6.7,1.5,10.0,NaN,1015.5,29.0,27.0,29.0,1.0,160
4,2021-09-10 18:00:00,-33.1,-37.5,100.0,9.3,NaN,NaN,NaN,1013.0,27.0,26.0,23.0,NaN,NaN


In [161]:
df.date_time.dt.hour

0     19
1     19
2     19
3     19
4     19
      ..
80    19
81    19
82    19
83    19
84    19
Name: date_time, Length: 85, dtype: int64

In [158]:
df.date_time.dt.hour = df.hour.values

ValueError: modifications to a property of a datetimelike object are not supported. Change values on the original.

In [115]:
df.hour

0     18.0
1     18.0
2     18.0
3     18.0
4     17.0
      ... 
87     6.0
88     6.0
89     6.0
90     6.0
91     6.0
Name: hour, Length: 92, dtype: float64